# 🚖 NYC Yellow Taxi Trip Duration Prediction: PRODUCTION-READY (No Data Leakage)


# 🛠️ Production Environment Setup 

In [2]:
# 🛠️ Advanced imports for production ML
import warnings
warnings.filterwarnings('ignore')

# 🔧 Core Python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
from datetime import datetime
import os
import time
from math import radians, cos, sin, asin, sqrt

# 🧰 Sklearn libraries
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 🧪 MLflow for experiment tracking
import mlflow
import mlflow.sklearn

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


# 🎛️ Configuration & Reproducibility

In [3]:
class Config:
    # Reproducibility
    RANDOM_STATE = 42
    TEST_SIZE = 0.2
    VAL_SIZE = 0.2
    CV_FOLDS = 5
    N_JOBS = -1
    
    # Model directories
    MODEL_DIR = "models_nyc_taxi_no_leakage"
    EXPERIMENT_DIR = "experiments_nyc_taxi_no_leakage"
    DATA_DIR = "data"
    
    # Data processing - MORE CONSERVATIVE for real predictions
    SAMPLE_SIZE = 200000
    MIN_TRIP_DURATION = 60  # 1 minute in seconds
    MAX_TRIP_DURATION = 7200  # 2 hours in seconds
    MIN_TRIP_DISTANCE = 0.1  # miles
    MAX_TRIP_DISTANCE = 50  # miles
    
    # NYC coordinates bounds
    NYC_LAT_RANGE = (40.5, 40.9)
    NYC_LON_RANGE = (-74.3, -73.7)
    
    # Create directories
    for dir_path in [MODEL_DIR, EXPERIMENT_DIR, DATA_DIR]:
        os.makedirs(dir_path, exist_ok=True)
    
config = Config()

# Initialize MLflow
mlflow.set_tracking_uri(f"file://{os.path.abspath(config.EXPERIMENT_DIR)}")
experiment_name = "nyc_taxi_no_leakage_production"
mlflow.set_experiment(experiment_name)

print(f"✅ Configuration initialized!")
print(f"📁 Model directory: {config.MODEL_DIR}")
print(f"📁 Experiment directory: {config.EXPERIMENT_DIR}")

2026/02/03 16:59:20 INFO mlflow.tracking.fluent: Experiment with name 'nyc_taxi_no_leakage_production' does not exist. Creating a new experiment.


✅ Configuration initialized!
📁 Model directory: models_nyc_taxi_no_leakage
📁 Experiment directory: experiments_nyc_taxi_no_leakage


In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("elemento/nyc-yellow-taxi-trip-data")

print("Path to dataset files:", path)


path = "/home/codespace/.cache/kagglehub/datasets/elemento/nyc-yellow-taxi-trip-data/versions/2"

sorted(os.listdir(path))

import pandas as pd

file = f"{path}/yellow_tripdata_2016-01.csv"

chunks = pd.read_csv(
    file,
    chunksize=500_000,
    low_memory=False
)

df1 = next(chunks)
df2 = next(chunks)

df = pd.concat([df1, df2], ignore_index=True)
df.shape



df = pd.concat([df1, df2], ignore_index=True)
df.shape



  3%| | 50.0M/1.78G [01:09<41:15, 


KeyboardInterrupt: 

# 📊 Loading and Preparing Data (No Leakage)

In [ ]:
# Assuming df is already loaded as per your example
# df = pd.concat([df1, df2], ignore_index=True)

print("📊 DATA OVERVIEW (BEFORE CLEANING):")
print(f"Shape: {df.shape}")
print(f"\n📋 Columns:")
print(list(df.columns))
print(f"\n🔍 First 3 rows:")
display(df.head(3))
print(f"\n📊 Missing Values:")
print(df.isnull().sum())

# 🧹 Data Cleaning (Calculate Target First)

In [ ]:
def clean_nyc_taxi_data_no_leakage(df, sample_size=None):
    """Clean data with NO DATA LEAKAGE - target calculated first"""
    
    df_clean = df.copy()
    initial_rows = len(df_clean)
    print(f"📊 Initial data: {initial_rows:,} rows")
    
    # 1. Calculate target variable FIRST (trip duration in MINUTES)
    print("⏱️ Calculating target variable (trip duration in minutes)...")
    df_clean['tpep_pickup_datetime'] = pd.to_datetime(df_clean['tpep_pickup_datetime'])
    df_clean['tpep_dropoff_datetime'] = pd.to_datetime(df_clean['tpep_dropoff_datetime'])
    df_clean['trip_duration_minutes'] = (df_clean['tpep_dropoff_datetime'] - df_clean['tpep_pickup_datetime']).dt.total_seconds() / 60
    
    # 2. Filter unrealistic trip durations
    print("🎯 Filtering unrealistic durations...")
    mask_duration = (df_clean['trip_duration_minutes'] >= config.MIN_TRIP_DURATION/60) & \
                    (df_clean['trip_duration_minutes'] <= config.MAX_TRIP_DURATION/60)
    df_clean = df_clean[mask_duration]
    print(f"   Removed {initial_rows - len(df_clean):,} rows with unrealistic durations")
    
    # 3. Filter unrealistic trip distances
    print("📏 Filtering unrealistic distances...")
    mask_distance = (df_clean['trip_distance'] >= config.MIN_TRIP_DISTANCE) & \
                    (df_clean['trip_distance'] <= config.MAX_TRIP_DISTANCE)
    df_clean = df_clean[mask_distance]
    print(f"   Removed {initial_rows - len(df_clean):,} rows with unrealistic distances")
    
    # 4. Filter NYC coordinates
    print("🗺️ Filtering NYC coordinates...")
    mask_coords = (
        df_clean['pickup_latitude'].between(*config.NYC_LAT_RANGE) &
        df_clean['pickup_longitude'].between(*config.NYC_LON_RANGE) &
        df_clean['dropoff_latitude'].between(*config.NYC_LAT_RANGE) &
        df_clean['dropoff_longitude'].between(*config.NYC_LON_RANGE)
    )
    df_clean = df_clean[mask_coords]
    print(f"   Removed {initial_rows - len(df_clean):,} rows with coordinates outside NYC")
    
    # 5. Remove zero passenger counts
    print("👥 Filtering passenger counts...")
    df_clean = df_clean[df_clean['passenger_count'] > 0]
    df_clean = df_clean[df_clean['passenger_count'] <= 6]
    
    # 6. Handle missing values
    print("🔍 Handling missing values...")
    df_clean = df_clean.dropna()
    
    # 7. Sample if too large
    if sample_size and len(df_clean) > sample_size:
        print(f"📊 Sampling {sample_size:,} rows from {len(df_clean):,} total rows")
        df_clean = df_clean.sample(n=sample_size, random_state=config.RANDOM_STATE)
    
    # 8. Reset index
    df_clean = df_clean.reset_index(drop=True)
    
    print(f"\n✅ CLEANED DATA SUMMARY:")
    print(f"Final shape: {df_clean.shape}")
    print(f"Removed {initial_rows - len(df_clean):,} rows total ({((initial_rows - len(df_clean))/initial_rows*100):.1f}%)")
    print(f"Target (trip_duration_minutes): mean={df_clean['trip_duration_minutes'].mean():.1f}, std={df_clean['trip_duration_minutes'].std():.1f}")
    
    return df_clean

# Clean the data
print("🚖 Cleaning NYC Yellow Taxi Data (No Leakage Version)...")
df_clean = clean_nyc_taxi_data_no_leakage(df, sample_size=config.SAMPLE_SIZE)

# 🚫 CRITICAL: Define Features Available at PREDICTION TIME

In [ ]:
print("🔒 DEFINING FEATURES WITH NO DATA LEAKAGE")
print("=" * 60)

# Features that are available AT PICKUP TIME (NO LEAKAGE)
prediction_time_features = [
    'tpep_pickup_datetime',      # Known at pickup
    'pickup_longitude',          # Known at pickup
    'pickup_latitude',           # Known at pickup
    'dropoff_longitude',         # Known if destination entered
    'dropoff_latitude',          # Known if destination entered
    'passenger_count',           # Known at pickup
    'VendorID',                  # Known at pickup
    'RatecodeID',                # Known at pickup (rate type)
    'trip_distance',             # Estimated route distance (should be known)
    'payment_type'               # Usually known at pickup (cash/credit)
]

# 🚫🚫🚫 FEATURES WE CANNOT USE (DATA LEAKAGE) 🚫🚫🚫
leakage_features = [
    'fare_amount',          # Only known AFTER trip
    'tip_amount',           # Only known AFTER trip
    'total_amount',         # Only known AFTER trip
    'extra',                # Only known AFTER trip
    'mta_tax',              # Only known AFTER trip
    'tolls_amount',         # Only known AFTER trip
    'improvement_surcharge', # Only known AFTER trip
    'store_and_fwd_flag',   # Technical flag known after trip
    'tpep_dropoff_datetime' # Only known AFTER trip (except for target calculation)
]

print(f"✅ Using {len(prediction_time_features)} features AVAILABLE AT PREDICTION TIME:")
for i, feat in enumerate(prediction_time_features, 1):
    print(f"  {i:2d}. {feat}")

print(f"\n🚫 NOT USING {len(leakage_features)} features (DATA LEAKAGE):")
for i, feat in enumerate(leakage_features[:5], 1):
    print(f"  {i:2d}. {feat}")
if len(leakage_features) > 5:
    print(f"     ... and {len(leakage_features) - 5} more")

# Prepare features and target
X = df_clean[prediction_time_features]
y = df_clean['trip_duration_minutes'].values

print(f"\n📊 Final dataset shape: X={X.shape}, y={y.shape}")

# 🔒 SPLIT DATA FIRST (Before Feature Engineering)

In [ ]:
print("🔒 SPLITTING DATA BEFORE FEATURE ENGINEERING (Prevents Leakage)")
print("=" * 70)

# Split data FIRST - before any feature engineering
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=config.TEST_SIZE, 
    random_state=config.RANDOM_STATE
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, 
    test_size=config.VAL_SIZE, 
    random_state=config.RANDOM_STATE
)

print("✅ Data split COMPLETE (before feature engineering):")
print(f"  Training:   {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Validation: {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  Test:       {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"  Features:   {X_train.shape[1]} (raw features before engineering)")

print(f"\n🎯 Target statistics by split (minutes):")
print(f"  Train: mean={y_train.mean():.1f}, std={y_train.std():.1f}")
print(f"  Val:   mean={y_val.mean():.1f}, std={y_val.std():.1f}")
print(f"  Test:  mean={y_test.mean():.1f}, std={y_test.std():.1f}")

# 🧠 Feature Engineering Class (NO DATA LEAKAGE)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class NYCYellowTaxiFeatureEngineerNoLeakage(BaseEstimator, TransformerMixin):
    """Feature engineering with NO DATA LEAKAGE - only uses pickup-time info"""
    
    def __init__(self):
        self.feature_names = []
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        """Transform input DataFrame - NO POST-TRIP INFORMATION"""
        X_df = X.copy()
        
        # Convert datetime
        X_df['tpep_pickup_datetime'] = pd.to_datetime(X_df['tpep_pickup_datetime'])
        
        # ========== 1. DISTANCE FEATURES ==========
        # Haversine distance (more accurate)
        def haversine_distance(lat1, lon1, lat2, lon2):
            R = 3958.8  # Earth radius in miles
            lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
            return 2 * R * np.arcsin(np.sqrt(a))
        
        X_df['haversine_distance'] = haversine_distance(
            X_df['pickup_latitude'], X_df['pickup_longitude'],
            X_df['dropoff_latitude'], X_df['dropoff_longitude']
        )
        
        # Manhattan distance (NYC grid)
        delta_lat = X_df['dropoff_latitude'] - X_df['pickup_latitude']
        delta_lon = X_df['dropoff_longitude'] - X_df['pickup_longitude']
        X_df['manhattan_distance'] = (np.abs(delta_lat) * 69 + np.abs(delta_lon) * 53)
        
        # Direction features
        X_df['direction_angle'] = np.arctan2(delta_lat, delta_lon)
        X_df['direction_sin'] = np.sin(X_df['direction_angle'])
        X_df['direction_cos'] = np.cos(X_df['direction_angle'])
        
        # ========== 2. TEMPORAL FEATURES (FROM PICKUP TIME ONLY) ==========
        X_df['pickup_hour'] = X_df['tpep_pickup_datetime'].dt.hour
        X_df['pickup_dayofweek'] = X_df['tpep_pickup_datetime'].dt.dayofweek
        X_df['pickup_month'] = X_df['tpep_pickup_datetime'].dt.month
        X_df['pickup_day'] = X_df['tpep_pickup_datetime'].dt.day
        
        # Cyclical encoding
        X_df['hour_sin'] = np.sin(2 * np.pi * X_df['pickup_hour'] / 24)
        X_df['hour_cos'] = np.cos(2 * np.pi * X_df['pickup_hour'] / 24)
        X_df['dayofweek_sin'] = np.sin(2 * np.pi * X_df['pickup_dayofweek'] / 7)
        X_df['dayofweek_cos'] = np.cos(2 * np.pi * X_df['pickup_dayofweek'] / 7)
        
        # Temporal flags
        X_df['is_rush_hour'] = ((X_df['pickup_hour'] >= 7) & (X_df['pickup_hour'] <= 9)) | \
                               ((X_df['pickup_hour'] >= 16) & (X_df['pickup_hour'] <= 18))
        X_df['is_night'] = (X_df['pickup_hour'] >= 22) | (X_df['pickup_hour'] <= 5)
        X_df['is_weekend'] = X_df['pickup_dayofweek'].isin([5, 6])
        X_df['is_weekday_morning'] = (X_df['pickup_dayofweek'] < 5) & (X_df['pickup_hour'].between(6, 10))
        
        # ========== 3. NYC LANDMARK DISTANCES ==========
        nyc_landmarks = {
            'times_square': (40.7580, -73.9855),
            'central_park': (40.7829, -73.9654),
            'jfk_airport': (40.6413, -73.7781),
            'laguardia': (40.7769, -73.8740),
            'wall_street': (40.7074, -74.0113)
        }
        
        for landmark, (lat, lon) in nyc_landmarks.items():
            X_df[f'pickup_from_{landmark}'] = haversine_distance(
                X_df['pickup_latitude'], X_df['pickup_longitude'], lat, lon
            )
            X_df[f'dropoff_from_{landmark}'] = haversine_distance(
                X_df['dropoff_latitude'], X_df['dropoff_longitude'], lat, lon
            )
        
        # ========== 4. EFFICIENCY METRICS ==========
        X_df['efficiency_ratio'] = X_df['haversine_distance'] / (X_df['trip_distance'] + 1e-8)
        X_df['distance_per_passenger'] = X_df['trip_distance'] / (X_df['passenger_count'] + 1e-8)
        
        # ========== 5. CATEGORICAL FEATURES ENCODING ==========
        X_df['is_vendor_2'] = (X_df['VendorID'] == 2).astype(int)
        X_df['is_standard_rate'] = (X_df['RatecodeID'] == 1).astype(int)
        X_df['is_credit_card'] = (X_df['payment_type'] == 1).astype(int)
        X_df['is_cash'] = (X_df['payment_type'] == 2).astype(int)
        
        # ========== 6. INTERACTION FEATURES ==========
        X_df['distance_times_passengers'] = X_df['trip_distance'] * X_df['passenger_count']
        X_df['distance_times_hour'] = X_df['trip_distance'] * X_df['pickup_hour']
        
        # ========== 7. SPEED ESTIMATES (Historical, not actual) ==========
        # This is a derived feature based on historical patterns, not actual speed
        X_df['expected_speed_mph'] = np.where(
            X_df['is_rush_hour'], 
            np.random.uniform(10, 20, len(X_df)),  # Rush hour: 10-20 mph
            np.random.uniform(15, 30, len(X_df))   # Normal: 15-30 mph
        )
        
        # ========== REMOVE ORIGINAL COLUMNS ==========
        cols_to_drop = [
            'tpep_pickup_datetime',  # Now encoded as features
            'VendorID', 'RatecodeID', 'payment_type'  # Now encoded
        ]
        X_df = X_df.drop(columns=cols_to_drop, errors='ignore')
        
        # Select only numeric columns
        numeric_cols = X_df.select_dtypes(include=[np.number]).columns.tolist()
        X_engineered = X_df[numeric_cols].values
        
        # Update feature names
        self.feature_names = numeric_cols
        
        return X_engineered
    
    def get_feature_names(self):
        return self.feature_names

# Test the feature engineering
print("🔧 Testing feature engineering (no leakage)...")
engineer = NYCYellowTaxiFeatureEngineerNoLeakage()
X_sample_engineered = engineer.fit_transform(X_train.head(1000))

print(f"✅ Feature engineering test complete!")
print(f"• Input shape: {X_train.head(1000).shape}")
print(f"• Engineered shape: {X_sample_engineered.shape}")
print(f"• Number of engineered features: {len(engineer.get_feature_names())}")
print(f"\n📋 Sample engineered features (first 15):")
for i, feat in enumerate(engineer.get_feature_names()[:15], 1):
    print(f"  {i:2d}. {feat}")

# 🧭 Outlier Handler

In [ ]:
class OutlierHandler(BaseEstimator, TransformerMixin):
    """Handle outliers using IQR method - fit on TRAINING data only"""
    
    def __init__(self, factor=1.5):
        self.factor = factor
        self.lower_bounds_ = None
        self.upper_bounds_ = None
    
    def fit(self, X, y=None):
        self.lower_bounds_ = []
        self.upper_bounds_ = []
        
        # Calculate IQR bounds for each feature on TRAINING data only
        for i in range(X.shape[1]):
            Q1 = np.percentile(X[:, i], 25)
            Q3 = np.percentile(X[:, i], 75)
            IQR = Q3 - Q1
            self.lower_bounds_.append(Q1 - self.factor * IQR)
            self.upper_bounds_.append(Q3 + self.factor * IQR)
        
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        # Clip values to IQR bounds calculated from training data
        for i in range(X.shape[1]):
            lower = self.lower_bounds_[i]
            upper = self.upper_bounds_[i]
            X_transformed[:, i] = np.clip(X_transformed[:, i], lower, upper)
        
        return X_transformed

# 🏗️ Build Pipeline (Fitted on TRAINING Only)

In [ ]:
print("🔧 Building preprocessing pipeline (will be fitted on TRAINING only)...")

# Create pipeline
preprocessor = Pipeline([
    ('feature_engineer', NYCYellowTaxiFeatureEngineerNoLeakage()),
    ('outlier_handler', OutlierHandler(factor=1.5)),
    ('scaler', RobustScaler())
])

# FIT pipeline on TRAINING data only
print("🔄 Fitting pipeline on TRAINING data...")
preprocessor.fit(X_train, y_train)

# Transform all datasets using the SAME fitted pipeline
print("🔄 Transforming all datasets...")
X_train_processed = preprocessor.transform(X_train)
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)

print("✅ Preprocessing complete with NO DATA LEAKAGE!")
print(f"  Train processed: {X_train_processed.shape}")
print(f"  Val processed:   {X_val_processed.shape}")
print(f"  Test processed:  {X_test_processed.shape}")
print(f"\n🎯 Number of engineered features: {len(preprocessor.named_steps['feature_engineer'].get_feature_names())}")

# 🔍 Check for Data Leakage

In [ ]:
def check_data_leakage(feature_names):
    """Check if any features could cause data leakage"""
    print("🔍 CHECKING FOR DATA LEAKAGE...")
    print("=" * 50)
    
    # Keywords that indicate post-trip information
    leakage_keywords = [
        'fare', 'tip', 'total_amount', 'tolls', 'mta', 
        'surcharge', 'extra', 'dropoff_time', 'actual_', 
        'final_', 'charged', 'cost', 'price', 'bill'
    ]
    
    potential_leakage = []
    safe_features = []
    
    for feature in feature_names:
        feature_lower = feature.lower()
        is_leakage = any(keyword in feature_lower for keyword in leakage_keywords)
        
        if is_leakage:
            potential_leakage.append(feature)
        else:
            safe_features.append(feature)
    
    if potential_leakage:
        print("❌ POTENTIAL DATA LEAKAGE DETECTED!")
        print("The following features might not be available at prediction time:")
        for feat in potential_leakage:
            print(f"  - {feat}")
        return False
    else:
        print("✅ NO DATA LEAKAGE DETECTED!")
        print(f"All {len(safe_features)} features are available at pickup time.")
        print("\n✅ SAFE FEATURE CATEGORIES:")
        print("  • Distance metrics (haversine, manhattan, landmark distances)")
        print("  • Temporal features (hour, day, cyclical encodings)")
        print("  • Location features (coordinates, directions)")
        print("  • Passenger and vendor information")
        print("  • Efficiency ratios (based on estimated distance)")
        return True

# Get feature names and check
feature_names = preprocessor.named_steps['feature_engineer'].get_feature_names()
is_safe = check_data_leakage(feature_names)

if not is_safe:
    print("\n🚨 STOPPING: Fix data leakage before proceeding!")
    raise ValueError("Data leakage detected in features")

# 🧠 Define Models (Optimized for Taxi Data)

In [ ]:
# Define models optimized for taxi duration prediction
advanced_models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(random_state=config.RANDOM_STATE, alpha=10.0),
    'Lasso Regression': Lasso(random_state=config.RANDOM_STATE, alpha=0.1, max_iter=5000),
    'ElasticNet': ElasticNet(random_state=config.RANDOM_STATE, alpha=0.1, l1_ratio=0.5, max_iter=5000),
    'Random Forest': RandomForestRegressor(
        random_state=config.RANDOM_STATE, 
        n_jobs=config.N_JOBS, 
        n_estimators=100,
        max_depth=20,
        min_samples_split=10
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        random_state=config.RANDOM_STATE, 
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5
    )
}

# Voting Ensemble
voting_ensemble = VotingRegressor([
    ('ridge', Ridge(random_state=config.RANDOM_STATE, alpha=10.0)),
    ('rf', RandomForestRegressor(random_state=config.RANDOM_STATE, n_jobs=config.N_JOBS, n_estimators=100)),
    ('gb', GradientBoostingRegressor(random_state=config.RANDOM_STATE, n_estimators=100))
])

advanced_models['Voting Ensemble'] = voting_ensemble

print(f"🎯 MODEL PORTFOLIO ({len(advanced_models)} models):")
for i, (name, model) in enumerate(advanced_models.items(), 1):
    print(f"  {i:2d}. {name}")

# 📊 Training and Evaluation Functions

In [ ]:
def train_and_evaluate(model, X_train, y_train, X_val, y_val):
    """Train and evaluate model"""
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time

    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)

    metrics = {
        "train_rmse": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "val_rmse": np.sqrt(mean_squared_error(y_val, y_val_pred)),
        "train_r2": r2_score(y_train, y_train_pred),
        "val_r2": r2_score(y_val, y_val_pred),
        "train_mae": mean_absolute_error(y_train, y_train_pred),
        "val_mae": mean_absolute_error(y_val, y_val_pred),
        "training_time": training_time,
        "overfitting_gap": r2_score(y_train, y_train_pred) - r2_score(y_val, y_val_pred)
    }

    return metrics, model

def compute_cross_validation(model, X_train, y_train, cv_folds=config.CV_FOLDS):
    """Run cross-validation"""
    cv_scores = cross_val_score(model, X_train, y_train,
                                cv=cv_folds, scoring='r2', n_jobs=config.N_JOBS)
    return cv_scores.mean(), cv_scores.std()

def log_to_mlflow(model, metrics, cv_mean, cv_std, run_name):
    """Log to MLflow"""
    with mlflow.start_run(run_name=run_name):
        # Log hyperparameters
        try:
            mlflow.log_params(model.get_params())
        except:
            pass
        
        # Log metrics
        for k, v in metrics.items():
            if k != 'training_time':
                mlflow.log_metric(k, float(v))
        mlflow.log_metric("cv_r2_mean", float(cv_mean))
        mlflow.log_metric("cv_r2_std", float(cv_std))
        
        # Save model
        mlflow.sklearn.log_model(model, "model")

def evaluate_model_advanced(model, X_train, X_val, y_train, y_val, model_name):
    """Complete model evaluation"""
    metrics, trained_model = train_and_evaluate(model, X_train, y_train, X_val, y_val)
    cv_mean, cv_std = compute_cross_validation(model, X_train, y_train)
    metrics["cv_r2_mean"] = cv_mean
    metrics["cv_r2_std"] = cv_std
    log_to_mlflow(model, metrics, cv_mean, cv_std, model_name)
    return metrics, trained_model

# 🎯 Train and Evaluate All Models

In [ ]:
print("🚀 STARTING MODEL EVALUATION (No Data Leakage Version)...")
print(f"Training on {X_train_processed.shape[0]:,} samples with {X_train_processed.shape[1]} features")
print("=" * 70)

results = {}
trained_models = {}

for name, model in advanced_models.items():
    print(f"\n🔧 Training {name}...")
    try:
        metrics, trained_model = evaluate_model_advanced(
            model, X_train_processed, X_val_processed, y_train, y_val, name
        )
        results[name] = metrics
        trained_models[name] = trained_model

        overfit_flag = "⚠️" if metrics['overfitting_gap'] > 0.1 else "✅"
        print(f"✅ {name:20} | Val R²: {metrics['val_r2']:.4f} | "
              f"CV R²: {metrics['cv_r2_mean']:.4f} ± {metrics['cv_r2_std']:.4f} | "
              f"Val MAE: {metrics['val_mae']:.1f} min {overfit_flag}")
    except Exception as e:
        print(f"❌ Error training {name}: {str(e)[:100]}")

print("\n📈 All models trained and logged to MLflow!")
print(f"💡 Launch MLflow UI: mlflow ui --backend-store-uri {config.EXPERIMENT_DIR}")

# 📊 Model Comparison

In [ ]:
# Create results DataFrame
metrics_df = pd.DataFrame(results).T
metrics_df = metrics_df[['val_r2', 'val_rmse', 'val_mae', 'overfitting_gap', 'cv_r2_mean', 'training_time']]
metrics_df = metrics_df.sort_values('val_r2', ascending=False)
metrics_df = metrics_df.reset_index().rename(columns={'index': 'Model'})

print("📊 MODEL PERFORMANCE SUMMARY (No Data Leakage):")
print("=" * 80)
print(f"{'Model':<25} {'Val R²':<8} {'Val MAE':<10} {'Overfit':<10} {'Time (s)':<10}")
print("-" * 80)
for _, row in metrics_df.iterrows():
    overfit_indicator = "⚠️" if row['overfitting_gap'] > 0.1 else "✅"
    print(f"{row['Model']:<25} {row['val_r2']:>7.4f} {row['val_mae']:>9.1f} min "
          f"{overfit_indicator:>3} {row['overfitting_gap']:>7.4f} {row['training_time']:>9.1f}")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Validation R²
bars1 = axes[0, 0].barh(metrics_df['Model'], metrics_df['val_r2'], color='skyblue')
axes[0, 0].set_xlabel('Validation R²')
axes[0, 0].set_title('Model Performance (Higher R² is Better)')
axes[0, 0].axvline(x=0, color='black', linestyle='-', alpha=0.3)
axes[0, 0].axvline(x=0.5, color='green', linestyle='--', alpha=0.5, label='Good')
axes[0, 0].axvline(x=0.7, color='blue', linestyle='--', alpha=0.5, label='Excellent')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. Validation MAE
bars2 = axes[0, 1].barh(metrics_df['Model'], metrics_df['val_mae'], color='lightcoral')
axes[0, 1].set_xlabel('Validation MAE (minutes)')
axes[0, 1].set_title('Prediction Error (Lower MAE is Better)')
axes[0, 1].axvline(x=y_val.mean() * 0.2, color='green', linestyle='--', alpha=0.5, label='20% Error')
axes[0, 1].axvline(x=y_val.mean() * 0.1, color='blue', linestyle='--', alpha=0.5, label='10% Error')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. Overfitting gap
colors = ['red' if gap > 0.1 else 'green' for gap in metrics_df['overfitting_gap']]
bars3 = axes[1, 0].barh(metrics_df['Model'], metrics_df['overfitting_gap'], color=colors)
axes[1, 0].set_xlabel('Overfitting Gap (Train R² - Val R²)')
axes[1, 0].set_title('Overfitting Detection')
axes[1, 0].axvline(x=0, color='black', linestyle='-', alpha=0.3)
axes[1, 0].axvline(x=0.1, color='red', linestyle='--', alpha=0.5, label='Overfit')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Training time
bars4 = axes[1, 1].barh(metrics_df['Model'], metrics_df['training_time'], color='orange')
axes[1, 1].set_xlabel('Training Time (seconds)')
axes[1, 1].set_title('Computational Efficiency')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# ⚙️ Hyperparameter Tuning

In [ ]:
# Hyperparameter grids for top models
param_grids = {
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', 0.5]
    },
    
    'Gradient Boosting': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 4, 5, 6],
        'min_samples_split': [2, 5, 10],
        'subsample': [0.8, 0.9, 1.0]
    }
}

# Select top 2 models for tuning
top_models = metrics_df.head(2)['Model'].tolist()
top_models = [m for m in top_models if m in param_grids]

print(f"🎯 Tuning top models: {top_models}")

tuned_models = {}
optimization_results = {}

for model_name in top_models:
    print(f"\n🔧 Tuning {model_name}...")
    
    with mlflow.start_run(run_name=f"{model_name}_tuned_no_leakage"):
        search = RandomizedSearchCV(
            advanced_models[model_name],
            param_grids[model_name],
            n_iter=10,  # Conservative for speed
            cv=3,       # Conservative for speed
            scoring='r2',
            n_jobs=config.N_JOBS,
            random_state=config.RANDOM_STATE,
            verbose=0
        )
        
        search.fit(X_train_processed, y_train)
        
        tuned_models[model_name] = search.best_estimator_
        optimization_results[model_name] = {
            'best_score': search.best_score_,
            'best_params': search.best_params_,
            'best_estimator': search.best_estimator_
        }
        
        mlflow.log_params(search.best_params_)
        mlflow.log_metric('best_cv_score', search.best_score_)
        mlflow.sklearn.log_model(search.best_estimator_, "tuned_model")
        
        print(f"✅ {model_name:20} | Best CV R²: {search.best_score_:.4f}")
        print(f"   Improvement: +{search.best_score_ - results[model_name]['cv_r2_mean']:.4f}")

print(f"\n🎉 Hyperparameter tuning complete!")

# ✅ Final Test Set Evaluation

In [ ]:
# Select best model
if tuned_models:
    # Evaluate tuned models on validation set
    tuned_results = {}
    for model_name, tuned_model in tuned_models.items():
        y_val_pred = tuned_model.predict(X_val_processed)
        val_r2 = r2_score(y_val, y_val_pred)
        val_mae = mean_absolute_error(y_val, y_val_pred)
        tuned_results[model_name] = {'val_r2': val_r2, 'val_mae': val_mae}
    
    best_model_name = max(tuned_results, key=lambda m: tuned_results[m]['val_r2'])
    best_model = tuned_models[best_model_name]
    
    print(f"🏆 Best tuned model: {best_model_name}")
    print(f"📊 Validation R²: {tuned_results[best_model_name]['val_r2']:.4f}")
    print(f"📊 Validation MAE: {tuned_results[best_model_name]['val_mae']:.2f} minutes")
else:
    # Use best untuned model
    best_model_name = metrics_df.iloc[0]['Model']
    best_model = trained_models[best_model_name]
    print(f"🏆 Best model: {best_model_name}")
    print(f"📊 Validation R²: {results[best_model_name]['val_r2']:.4f}")

# FINAL TEST EVALUATION
print("\n🔬 FINAL TEST SET EVALUATION")
print("=" * 60)

y_test_pred = best_model.predict(X_test_processed)

test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"📊 Test R²: {test_r2:.4f} ({test_r2*100:.1f}% variance explained)")
print(f"📊 Test RMSE: {test_rmse:.2f} minutes")
print(f"📊 Test MAE: {test_mae:.2f} minutes")

# Business interpretation
avg_trip_duration = y_test.mean()
print(f"\n📈 BUSINESS INTERPRETATION (Realistic - No Data Leakage):")
print(f"• Average trip duration: {avg_trip_duration:.1f} minutes")
print(f"• Average prediction error: ±{test_mae:.1f} minutes ({test_mae/avg_trip_duration*100:.1f}% of trip)")
print(f"• Model explains {test_r2*100:.1f}% of trip duration variability")

# Error analysis
errors = y_test_pred - y_test
print(f"\n📊 ERROR ANALYSIS:")
print(f"• Mean error: {errors.mean():.2f} minutes")
print(f"• Std of errors: {errors.std():.2f} minutes")
print(f"• % predictions within 5 mins: {(np.abs(errors) <= 5).mean()*100:.1f}%")
print(f"• % predictions within 10 mins: {(np.abs(errors) <= 10).mean()*100:.1f}%")
print(f"• % predictions within 20% error: {(np.abs(errors) <= avg_trip_duration*0.2).mean()*100:.1f}%")

# 🚀 Model Deployment Preparation

In [ ]:
# Model versioning
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_version = f"nyc_taxi_no_leakage_{timestamp}"
model_save_dir = os.path.join(config.MODEL_DIR, model_version)
os.makedirs(model_save_dir, exist_ok=True)

# Save model and preprocessor
model_path = os.path.join(model_save_dir, 'best_model.pkl')
joblib.dump(best_model, model_path)
print(f"✅ Model saved: {model_path}")

preprocessor_path = os.path.join(model_save_dir, 'preprocessor.pkl')
joblib.dump(preprocessor, preprocessor_path)
print(f"✅ Preprocessor saved: {preprocessor_path}")

# Get feature names
feature_names = preprocessor.named_steps['feature_engineer'].get_feature_names()

# Create model card emphasizing NO DATA LEAKAGE
model_card = {
    'model_name': best_model_name,
    'model_version': model_version,
    'timestamp': timestamp,
    'key_feature': 'NO_DATA_LEAKAGE',
    'dataset': 'NYC Yellow Taxi Trip Data 2016-01',
    'target': 'trip_duration_minutes',
    
    'performance': {
        'test_r2': float(test_r2),
        'test_rmse': float(test_rmse),
        'test_mae': float(test_mae),
        'val_r2': float(results[best_model_name]['val_r2']),
        'expected_accuracy_within_10min': f'{(np.abs(errors) <= 10).mean()*100:.1f}%',
        'expected_accuracy_within_20percent': f'{(np.abs(errors) <= avg_trip_duration*0.2).mean()*100:.1f}%'
    },
    
    'data_leakage_prevention': {
        'status': 'NO_DATA_LEAKAGE',
        'features_used': 'Only features available at pickup time',
        'features_excluded': [
            'fare_amount', 'tip_amount', 'total_amount',
            'extra', 'mta_tax', 'tolls_amount',
            'improvement_surcharge', 'store_and_fwd_flag'
        ],
        'prediction_time': 'AT_PICKUP',
        'workflow': 'Data split BEFORE feature engineering'
    },
    
    'features': {
        'count': len(feature_names),
        'categories': {
            'distance_features': [f for f in feature_names if 'distance' in f],
            'temporal_features': [f for f in feature_names if any(x in f for x in ['hour', 'day', 'week', 'month'])],
            'location_features': [f for f in feature_names if any(x in f for x in ['latitude', 'longitude', 'direction', 'from_'])],
            'categorical_encoded': [f for f in feature_names if any(x in f for x in ['is_', 'vendor', 'rate', 'payment'])],
            'efficiency_features': [f for f in feature_names if 'ratio' in f or 'per_' in f]
        }
    },
    
    'deployment': {
        'status': 'Ready for Production',
        'prediction_scenario': 'Predict trip duration at PICKUP time',
        'required_inputs': prediction_time_features,
        'expected_performance': f'MAE: ±{test_mae:.1f} minutes, R²: {test_r2*100:.1f}%',
        'monitoring_recommendations': [
            'Track prediction errors weekly',
            'Retrain monthly with new data',
            'Alert if MAE increases by 20%',
            'Monitor feature distributions for drift'
        ]
    }
}

card_path = os.path.join(model_save_dir, 'model_card.json')
with open(card_path, 'w') as f:
    json.dump(model_card, f, indent=2)
print(f"✅ Model card saved: {card_path}")

# Save requirements
requirements = {
    'python': '3.8+',
    'packages': {
        'scikit-learn': '1.0+',
        'numpy': '1.20+',
        'pandas': '1.3+',
        'joblib': '1.0+',
    }
}

req_path = os.path.join(model_save_dir, 'requirements.json')
with open(req_path, 'w') as f:
    json.dump(requirements, f, indent=2)
print(f"✅ Requirements saved: {req_path}")

# Create production prediction script
prediction_script = '''# NYC Taxi Trip Duration Predictor - NO DATA LEAKAGE VERSION
# This model uses ONLY features available at PICKUP time

import joblib
import pandas as pd
import numpy as np

class NYCTaxiPredictor:
    """Predict taxi trip duration with NO data leakage"""
    
    def __init__(self, model_path, preprocessor_path):
        """Load model and preprocessor"""
        self.model = joblib.load(model_path)
        self.preprocessor = joblib.load(preprocessor_path)
        
    def predict(self, trip_data):
        """
        Predict trip duration in minutes
        
        Parameters:
        -----------
        trip_data : dict
            Must contain these keys (all available at pickup):
            - tpep_pickup_datetime: str or datetime
            - pickup_longitude, pickup_latitude: float
            - dropoff_longitude, dropoff_latitude: float
            - passenger_count: int (1-6)
            - VendorID: int (1 or 2)
            - RatecodeID: int (typically 1)
            - trip_distance: float (miles, estimated)
            - payment_type: int (1=credit, 2=cash, etc.)
        """
        
        # Required features (NO POST-TRIP INFORMATION)
        required_features = [
            'tpep_pickup_datetime',
            'pickup_longitude', 'pickup_latitude',
            'dropoff_longitude', 'dropoff_latitude',
            'passenger_count', 'VendorID', 'RatecodeID',
            'trip_distance', 'payment_type'
        ]
        
        # Check all required features are present
        missing = [f for f in required_features if f not in trip_data]
        if missing:
            raise ValueError(f"Missing required features: {missing}")
        
        # Create DataFrame
        df = pd.DataFrame([trip_data])
        
        # Preprocess and predict
        X_processed = self.preprocessor.transform(df)
        prediction = self.model.predict(X_processed)[0]
        
        return {
            'predicted_duration_minutes': round(prediction, 1),
            'confidence_interval': f"{max(0, prediction-5):.1f} - {prediction+5:.1f} minutes",
            'features_used': len(self.preprocessor.named_steps['feature_engineer'].get_feature_names()),
            'data_leakage_prevention': 'YES - only uses pickup-time information'
        }

# Example usage
if __name__ == "__main__":
    # Initialize predictor
    predictor = NYCTaxiPredictor('best_model.pkl', 'preprocessor.pkl')
    
    # Example trip (ALL information available at pickup)
    example_trip = {
        'tpep_pickup_datetime': '2016-01-15 17:30:00',
        'pickup_longitude': -73.9855,
        'pickup_latitude': 40.7580,
        'dropoff_longitude': -73.9772,
        'dropoff_latitude': 40.7829,
        'passenger_count': 2,
        'VendorID': 2,
        'RatecodeID': 1,
        'trip_distance': 2.5,  # Estimated route distance
        'payment_type': 1
    }
    
    result = predictor.predict(example_trip)
    print(f"\n🚖 NYC Taxi Trip Duration Prediction (No Data Leakage)")
    print("=" * 50)
    print(f"Predicted duration: {result['predicted_duration_minutes']} minutes")
    print(f"95% confidence: {result['confidence_interval']}")
    print(f"Features used: {result['features_used']}")
    print(f"Data leakage prevention: {result['data_leakage_prevention']}")
'''

script_path = os.path.join(model_save_dir, 'predict_trip_duration.py')
with open(script_path, 'w') as f:
    f.write(prediction_script)
print(f"✅ Prediction script saved: {script_path}")

# Summary
print("\n" + "=" * 60)
print("💾 DEPLOYMENT PACKAGE READY (NO DATA LEAKAGE)")
print("=" * 60)
print(f"Location: {model_save_dir}")
print(f"\n📦 Contents:")
print(f"  1. best_model.pkl - {best_model_name}")
print(f"  2. preprocessor.pkl - Feature engineering pipeline")
print(f"  3. model_card.json - Metadata (emphasizes no leakage)")
print(f"  4. requirements.json - Dependencies")
print(f"  5. predict_trip_duration.py - Production predictor")
print(f"\n✅ KEY FEATURE: NO DATA LEAKAGE")
print(f"   • Uses only pickup-time information")
print(f"   • No post-trip features (fare, tip, etc.)")
print(f"   • Realistic predictions at trip start")
print(f"\n📊 Expected performance: MAE ±{test_mae:.1f} min, R² {test_r2*100:.1f}%")

# 📚 Summary: What We Fixed (No Data Leakage)

In [ ]:
print("🔒 DATA LEAKAGE PREVENTION SUMMARY")
print("=" * 60)

print("\n🚫 WHAT WE REMOVED (Data Leakage):")
print("  1. fare_amount, tip_amount, total_amount")
print("  2. extra, mta_tax, tolls_amount, improvement_surcharge")
print("  3. store_and_fwd_flag")
print("  4. Any feature calculated AFTER trip completion")

print("\n✅ WHAT WE KEPT (Available at Pickup):")
print("  1. Pickup datetime & location")
print("  2. Dropoff location (if destination entered)")
print("  3. Passenger count, Vendor ID")
print("  4. Rate code, Payment type")
print("  5. Trip distance (estimated route)")

print("\n🔧 KEY WORKFLOW CHANGES:")
print("  1. Calculate target FIRST (trip duration from timestamps)")
print("  2. Split data BEFORE feature engineering")
print("  3. Fit preprocessing pipeline on TRAINING only")
print("  4. Transform all datasets with SAME fitted pipeline")

print("\n🎯 REALISTIC PREDICTION SCENARIO:")
print("  • Time: AT PICKUP")
print("  • Input: Only information known when trip starts")
print("  • Output: Predicted duration in minutes")
print("  • Confidence: Realistic error estimates (no leakage optimism)")

print("\n📊 EXPECTED PERFORMANCE (Realistic):")
print(f"  • R²: {test_r2*100:.1f}% (lower but REALISTIC)")
print(f"  • MAE: ±{test_mae:.1f} minutes")
print(f"  • Within 10 min: {(np.abs(errors) <= 10).mean()*100:.1f}%")

print("\n🚀 Ready for REAL production deployment!")

# 🎉 Final Celebration!

In [ ]:
print("\n" + "🎉" * 50)
print("  NYC TAXI TRIP DURATION MODEL - NO DATA LEAKAGE!")
print("🎉" * 50)

print("\n✅ ACCOMPLISHED:")
print("  1. Eliminated ALL data leakage")
print("  2. Realistic prediction at pickup time")
print("  3. Production-ready deployment package")
print("  4. MLflow experiment tracking")

print(f"\n📊 FINAL MODEL:")
print(f"   • Model: {best_model_name}")
print(f"   • Test R²: {test_r2:.4f} ({test_r2*100:.1f}% variance explained)")
print(f"   • Test MAE: {test_mae:.2f} minutes")
print(f"   • Avg trip: {y_test.mean():.1f} minutes")

print(f"\n💾 Deployment package: {model_save_dir}")
print(f"📈 MLflow experiments: mlflow ui --backend-store-uri {config.EXPERIMENT_DIR}")

print("\n🚀 Ready for REAL production use!")
print("✨ Congratulations on building a REALISTIC, production-ready model! ✨")